### Import Pydeseq2 Packages

In [ ]:
import os
import pickle as pkl

from pydeseq2.dds import DeseqDataSet
from pydeseq2.ds import DeseqStats
from pydeseq2.utils import load_example_data

In [ ]:
#load data, aka the two data_sets I create based on the cluster

### Data Filtering

In [ ]:
sample_to_keep = ~clinical_df.cluster.isna()
counts_df = counts_df.loc[sample_to_keep]
clinical_df = clinical_df.loc[sample_to_keep]

This filters out data that has NaN entries.

In [ ]:
genes_to_keep = counts_df.columns[counts_df.sum(axis=0) >= 10]
counts_df = counts_df[genes_to_keep]

Filters out genes that have less than 10 read counts in total.

### Single Factor Analysis

In [ ]:
counts_df = counts_df.astype(int)
dds = DeseqDataSet(counts=counts_df, clinical=clinical_df, design_factors='cluster', refit_cooks=True)
dds.deseq2()

Fitting size factors...
... done in 24.76 seconds.

Fitting dispersions...
... done in 107.44 seconds.

Fitting dispersion trend curve...
... done in 20.95 seconds.

Fitting MAP dispersions...
... done in 462.04 seconds.

Fitting LFCs...
... done in 583.82 seconds.

Refitting 0 outliers.



Initializes DeseqDataSet and fits disperisions and log2 fold change.

In [ ]:
print(dds.varm["LFC"])

           intercept  cluster_1_vs_0
5_8S_rRNA   2.256494       -0.069614
5S_rRNA     2.258279       -0.025425
7SK         2.557627        0.100134
A1BG        2.434840        0.213063
A1BG-AS1    2.452609        0.217494
...              ...             ...
ZYG11B      2.711379        0.147216
ZYX         2.886266        0.073090
ZYXP1       2.256494       -0.071036
ZZEF1       2.779507        0.100290
ZZZ3        2.735633        0.124306

[55691 rows x 2 columns]


### Statistical Analysis

In [ ]:
print(dds)
stat_res = DeseqStats(dds)
stat_res.summary()

AnnData object with n_obs × n_vars = 3218 × 55691
    obs: 'cluster'
    uns: 'trend_coeffs', '_squared_logres', 'prior_disp_var'
    obsm: 'design_matrix', 'size_factors', 'replaceable'
    varm: 'non_zero', '_rough_dispersions', 'genewise_dispersions', '_genewise_converged', '_normed_means', 'fitted_dispersions', 'MAP_dispersions', '_MAP_converged', 'dispersions', '_outlier_genes', 'LFC', '_LFC_converged', 'replaced'
    layers: 'normed_counts', '_mu_hat', '_mu_LFC', '_hat_diagonals', 'cooks'
Running Wald tests...
... done in 684.88 seconds.

Log2 fold change & Wald test p-value: cluster 1 vs 0


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
5_8S_rRNA,9.005096,-0.100432,0.022359,-4.491754,7.063892e-06,1.513350e-05
5S_rRNA,9.362858,-0.036681,0.022255,-1.648207,9.931027e-02,1.143460e-01
7SK,14.048487,0.144462,0.018971,7.614960,2.637724e-14,1.026394e-13
A1BG,13.697945,0.307384,0.020009,15.362405,2.925170e-53,1.147223e-51
A1BG-AS1,13.997978,0.313778,0.019826,15.826685,2.036728e-56,9.261697e-55
...,...,...,...,...,...,...
ZYG11B,17.071692,0.212388,0.017506,12.132249,7.126500e-34,1.038143e-32
ZYX,19.080679,0.105447,0.016129,6.537509,6.255191e-11,2.049887e-10
ZYXP1,8.994382,-0.102483,0.022362,-4.582913,4.585433e-06,1.113148e-05
ZZEF1,17.550189,0.144688,0.016979,8.521816,1.570714e-17,7.445918e-17


In [ ]:
stat_res.lfc_shrink(coeff="cluster_1_vs_0")
print(stat_res.shrunk_LFCs)
stat_res_1_vs_0 = DeseqStats(dds, contrast=["cluster", '1', '0'], n_cpus=8)
stat_res_1_vs_0.summary()

Fitting MAP LFCs...
... done in 16.36 seconds.

Shrunk Log2 fold change & Wald test p-value: cluster 1 vs 0


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
5_8S_rRNA,9.005096,-0.14427,0.06606,-4.491754,7.063892e-06,1.513350e-05
5S_rRNA,9.362858,-0.14427,0.06606,-1.648207,9.931027e-02,1.143460e-01
7SK,14.048487,-0.14427,0.06606,7.614960,2.637724e-14,1.026394e-13
A1BG,13.697945,-0.14427,0.06606,15.362405,2.925170e-53,1.147223e-51
A1BG-AS1,13.997978,-0.14427,0.06606,15.826685,2.036728e-56,9.261697e-55
...,...,...,...,...,...,...
ZYG11B,17.071692,-0.14427,0.06606,12.132249,7.126500e-34,1.038143e-32
ZYX,19.080679,-0.14427,0.06606,6.537509,6.255191e-11,2.049887e-10
ZYXP1,8.994382,-0.14427,0.06606,-4.582913,4.585433e-06,1.113148e-05
ZZEF1,17.550189,-0.14427,0.06606,8.521816,1.570714e-17,7.445918e-17


True
Running Wald tests...
... done in 731.42 seconds.

Log2 fold change & Wald test p-value: cluster 1 vs 0


,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
5_8S_rRNA,9.005096,-0.100432,0.022359,-4.491754,7.063892e-06,1.513350e-05
5S_rRNA,9.362858,-0.036681,0.022255,-1.648207,9.931027e-02,1.143460e-01
7SK,14.048487,0.144462,0.018971,7.614960,2.637724e-14,1.026394e-13
A1BG,13.697945,0.307384,0.020009,15.362405,2.925170e-53,1.147223e-51
A1BG-AS1,13.997978,0.313778,0.019826,15.826685,2.036728e-56,9.261697e-55
...,...,...,...,...,...,...
ZYG11B,17.071692,0.212388,0.017506,12.132249,7.126500e-34,1.038143e-32
ZYX,19.080679,0.105447,0.016129,6.537509,6.255191e-11,2.049887e-10
ZYXP1,8.994382,-0.102483,0.022362,-4.582913,4.585433e-06,1.113148e-05
ZZEF1,17.550189,0.144688,0.016979,8.521816,1.570714e-17,7.445918e-17


Applying threshold to data.

In [ ]:
stat_res_1_vs_0_filtered = stat_res_1_vs_0.results_df[stat_res_1_vs_0.results_df['padj'] < 0.5]
stat_res_1_vs_0_filtered

,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj
5_8S_rRNA,9.005096,-0.100432,0.022359,-4.491754,7.063892e-06,1.513350e-05
5S_rRNA,9.362858,-0.036681,0.022255,-1.648207,9.931027e-02,1.143460e-01
7SK,14.048487,0.144462,0.018971,7.614960,2.637724e-14,1.026394e-13
A1BG,13.697945,0.307384,0.020009,15.362405,2.925170e-53,1.147223e-51
A1BG-AS1,13.997978,0.313778,0.019826,15.826685,2.036728e-56,9.261697e-55
...,...,...,...,...,...,...
ZYG11B,17.071692,0.212388,0.017506,12.132249,7.126500e-34,1.038143e-32
ZYX,19.080679,0.105447,0.016129,6.537509,6.255191e-11,2.049887e-10
ZYXP1,8.994382,-0.102483,0.022362,-4.582913,4.585433e-06,1.113148e-05
ZZEF1,17.550189,0.144688,0.016979,8.521816,1.570714e-17,7.445918e-17


Finding the three genes with the highest log fold changes.

In [ ]:
sorted_stat_1_vs_0 = stat_res_1_vs_0_filtered.iloc[:, 1].sort_values(ascending=False)
top_three_genes = sorted_stat_1_vs_0[:3].index
top_three_genes

Index(['LRP2', 'IRX3', 'HPN'], dtype='object')

LRP2 is only druggable by ligand based assesment, and is involved in five pathways. The first is the vitaming D metabolism, then also HDL remodeling, plasma lipoprotein clearance, plasma lipoprotein assembkly, remodeling, and clearance and finally the metabolism fo steroids. IRX3 is not druggable and is involved in the regulation of gene expression in beta cells. HPN is involved in three pathways, the signaling by MST1, MET receptor activation, and signaling by receptor Tyrosine Kinases. The gene is tractible by bioactive compounds, enzyme and has a druggable structure.